In [15]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
import json

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
def load_trained_model(model_path):
    model = keras.models.load_model(model_path)
    return model

In [18]:
model_path = '/content/drive/My Drive/Colab Notebooks/model/crane_parts_model.keras'
model = load_trained_model(model_path)

In [19]:
def preprocess_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

In [20]:
CONFIDENCE_THRESHOLD = 0.70

In [21]:
# Predict the class of the input image with confidence threshold
def predict_image(model, image_path, class_labels, threshold=0.70):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction, axis=1)[0]
    predicted_label = class_labels[str(predicted_class)]
    confidence = np.max(prediction)

    if confidence < threshold:
        return "uncertain", confidence
    return predicted_label, confidence

In [22]:
def load_class_labels(json_path):
    with open(json_path, 'r') as json_file:
        class_labels = json.load(json_file)
    return class_labels

# Load the class labels
json_path = '/content/drive/My Drive/Colab Notebooks/model/class_labels.json'
class_labels = load_class_labels(json_path)


In [23]:
print(class_labels)

{'0': 'bridge idler wheel', '1': 'drill', '2': 'hook', '3': 'motor', '4': 'radio control'}


In [32]:
image_path = '/content/drive/My Drive/Colab Notebooks/Predict Img/Capture.png'  # Replace with the path to your input image
predicted_label, confidence = predict_image(model, image_path, class_labels, threshold=CONFIDENCE_THRESHOLD)

1/1 [==============================] - 1s 911ms/step


In [33]:
if predicted_label == "uncertain":
    print(f'The model is uncertain about the image. Confidence: {confidence:.2f}')
else:
    print(f'The prediction of the input image is: {predicted_label} with confidence: {confidence:.2f}')

The prediction of the input image is: motor with confidence: 0.83
